In [10]:
# using Pkg
# Pkg.activate(".")

In [11]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("fourier_tensor_train.jl")
include("bp_fourier.jl");

In [12]:
seed = 4
rng = MersenneTwister(seed)

T = 5
# N = 3
# A = [0 1 1; 
#      1 0 0;
#      1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]
N = 10
A = [0 1 1 1 1 1 1 1 1 1;
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0; 
     1 0 0 0 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 20
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end

β = 1.0
J = 1
h = 0.0
m⁰ = 0.3
K = 100

svd_trunc=TruncBond(4)

A

10×10 Matrix{Int64}:
 0  1  1  1  1  1  1  1  1  1
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0

In [13]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
# ϕᵢ = [ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        # a.=rand.(rng)
        a.=1.0
    end
end
iterate!(bp, maxiter=1; svd_trunc)

bp_fourier = convert_msg_beliefs(ComplexF64, bp)
;

In [14]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#159#162", Float64}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(25) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 431706890138)), 0, 1, false, false, 1.739284953405492e9, 1.739284953405492e9, 1.739284953405492e9)), [[[1.6500000000000001, 1.5, 1.5, 1.5, 1.5, 1.5], [1.65, 1.6142391233933644, 1.529685225288444, 1.5226080940979088, 1.5058747506695043, 1.5044741757775915], [1.65, 1.6142391233933644, 1.529685225288444, 1.5226080940979088, 1.5058747506695043, 1.5044741757775915], [1.6500000000000001, 1.6142391233933646, 1.5296852252884436, 1.5226080940979088, 1.5058747506

In [15]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

In [16]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [17]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.0047969967605145936

(1, 6)

In [18]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        push!(er_μ, abs(marg_fourier[1]-marg[1]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.004648974461376382

(Indexed Edge 1 => 6 with index 5, 6)